## imports and setting up the environment

In [24]:
# database related imports
from pyannote.database import registry, FileFinder

# training related imports
from pyannote.audio import Pipeline, Model
from pyannote.audio import Inference
from pyannote.audio.tasks import SpeakerDiarization, Segmentation

# metrics related imports
from pyannote.metrics.diarization import DiarizationErrorRate

import os

huggingface_token = os.getenv("HUGGINGFACE_TOKEN")


## Data preparation

In [28]:
registry.load_database("database.yml")

protocol = registry.get_protocol("BP.SpeakerDiarization.VlaamseAudio", {"audio":FileFinder()})

for file in protocol.train():
   print(file["uri"])
   assert "annotation" in file
   assert "annotated" in file


file1
file2
file3
file4


C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyannote\database\registry.py:499: UserWarning: Replacing existing BP.SpeakerDiarization.VlaamseAudio protocol by the one defined in 'C:\Users\verho\Documents\School\2024-2025\Bacherlorproef\bp-stemmen-onderscheiden\pyannote\database.yml'.
  warnings.warn(


In [29]:
database = registry.get_database("BP")

database.get_tasks()

['SpeakerDiarization']

In [26]:
protocol

## pretrained pyannote pipeline

In [17]:
# get the model from the pipeline
pretrained_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=huggingface_token)


current error rate of the pipeline

In [20]:
metric = DiarizationErrorRate()

for file in protocol.test():
    print(file)
    file["pretrained pipeline"] = pretrained_pipeline(file)
    metric(file["annotation"], file["pretrained pipeline"], uem=file["annotated"])

print(f"Diarization error rate is {100 * abs(metric):.1f}% for the pretrained model")

Diarization error rate is 25.0% for the pretrained model


## train the model

In [ ]:
pretrained_segm_model = pretrained_pipeline.segmentation_model
output_dir = "./models"

task = Segmentation(
    protocol,
    # pretrained_segm_model.specifications.duration,
    vad_loss="bce",
    loss="bce"
)

TypeError: __str__ returned non-string (type NoneType)